In [1]:
import sys
sys.path.append('../')
import pandas as pd
import joblib
from helper_files.modelling_functions import get_data, run_optuna_trials, evaluate_models, get_data_config, get_best_model
from helper_files.modelling_configs import *
from helper_files.etv_model import ETVModel
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, confusion_matrix


/Users/bradeneberhard/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '../data/processed/data_splits_1003.jblb'
train_df, test_df_time, test_df_random, test_df_thrower = get_data(data_path)
test_dfs = [test_df_random, test_df_time, test_df_thrower]
storage = 'sqlite:///model_tuning.db'
train_df_cp = train_df[train_df['throw_distance'] >= 1.5]

In [3]:
fv_data_config = {'features': thrower_context+game_context, 'target':'point_outcome', 'train_df_raw':train_df, 'test_dfs_raw':test_dfs}
fv_best_models, fv_best_params, fv_data_config_updated = run_optuna_trials(fv_data_config, model_config, 'fv', delete_models=True, storage='sqlite:///model_tuning.db')
metrics_df = evaluate_models(fv_best_models, fv_data_config_updated)
metrics_df

[I 2024-11-19 15:35:41,442] A new study created in RDB with name: logreg_fv
[I 2024-11-19 15:35:46,956] Trial 0 finished with value: 0.6399506704157367 and parameters: {'noise_factor': 0.03679519025070552}. Best is trial 0 with value: 0.6399506704157367.
[I 2024-11-19 15:35:47,402] A new study created in RDB with name: xgb_fv
[I 2024-11-19 15:35:51,086] Trial 0 finished with value: 0.6586025976789589 and parameters: {'noise_factor': 0.0033304092952659847, 'n_estimators': 20, 'learning_rate': 0.003602312772607612, 'max_depth': 8}. Best is trial 0 with value: 0.6586025976789589.
[I 2024-11-19 15:35:57,694] Trial 1 finished with value: 0.6830917628611737 and parameters: {'noise_factor': 0.015195409759830575, 'n_estimators': 105, 'learning_rate': 0.25264259197026134, 'max_depth': 5}. Best is trial 1 with value: 0.6830917628611737.
[I 2024-11-19 15:36:05,207] Trial 2 finished with value: 0.6462352221987342 and parameters: {'noise_factor': 0.04596107259071772, 'n_estimators': 188, 'learning_

,Model,train_AUC,train_Accuracy,train_NPV,train_PPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_random_PPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_time_PPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV,test_df_thrower_PPV
0,logreg,0.639970,0.710329,0.566455,0.714495,0.640571,0.712444,0.584059,0.716029,0.640926,0.698326,0.543712,0.704515,0.631261,0.732633,0.554755,0.737441
1,xgb,0.791665,0.771229,0.772848,0.771012,0.791497,0.768593,0.765224,0.769029,0.786553,0.759659,0.761700,0.759371,0.789164,0.789436,0.772507,0.791518


In [4]:
cp_data_config = {'features': thrower_context+receiver_context+throw_context+game_context, 'target':'completion', 'train_df_raw':train_df_cp, 'test_dfs_raw':test_dfs}
cp_best_models, cp_best_params, cp_data_config_updated = run_optuna_trials(cp_data_config, model_config, 'cp', delete_models=True, storage='sqlite:///model_tuning.db')
metrics_df = evaluate_models(cp_best_models, cp_data_config_updated)
metrics_df


[I 2024-11-19 15:38:57,585] A new study created in RDB with name: logreg_cp
[I 2024-11-19 15:39:02,023] Trial 0 finished with value: 0.7881830735541134 and parameters: {'noise_factor': 0.034292292860955316}. Best is trial 0 with value: 0.7881830735541134.
[I 2024-11-19 15:39:02,509] A new study created in RDB with name: xgb_cp
[I 2024-11-19 15:39:11,256] Trial 0 finished with value: 0.7979763328893452 and parameters: {'noise_factor': 0.020695197185570793, 'n_estimators': 58, 'learning_rate': 0.0004973987009078766, 'max_depth': 8}. Best is trial 0 with value: 0.7979763328893452.
[I 2024-11-19 15:39:18,698] Trial 1 finished with value: 0.7977805321118063 and parameters: {'noise_factor': 0.011858589194128405, 'n_estimators': 48, 'learning_rate': 0.002090468172596488, 'max_depth': 7}. Best is trial 0 with value: 0.7979763328893452.
[I 2024-11-19 15:39:28,669] Trial 2 finished with value: 0.802389223288858 and parameters: {'noise_factor': 0.00836732627354424, 'n_estimators': 98, 'learning_r

,Model,train_AUC,train_Accuracy,train_NPV,train_PPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_random_PPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_time_PPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV,test_df_thrower_PPV
0,logreg,0.788291,0.937035,0.437584,0.943552,0.713851,0.931403,0.442781,0.937157,0.692978,0.932794,0.419922,0.940663,0.747254,0.935196,0.454545,0.942731
1,xgb,0.932378,0.952347,0.910358,0.953060,0.905513,0.945929,0.905199,0.946603,0.899860,0.948231,0.903108,0.948971,0.913358,0.950250,0.932692,0.950532


In [5]:
for model_name in ['logreg', 'xgb']:
    study_name=f'{model_name}_cp'
    target = 'completion'
    cp_model = cp_best_models[model_name]
    study_name=f'{model_name}_fv'
    target = 'point_outcome'
    fv_model = fv_best_models[model_name]
    etv_model = ETVModel(cp_model, fv_model)
    joblib.dump({'cp_model':cp_model, 'fv_model':fv_model, 'etv_model':etv_model}, f'../data/models/{model_name}_models_auc.jblb')

In [6]:
models = joblib.load('../data/models/xgb_models_auc.jblb')

In [13]:
fv_results, cp_results = [], []
fv_model = models['fv_model']['model']
fv_scaler = models['fv_model']['scaler']
cp_model = models['cp_model']['model']
cp_scaler = models['cp_model']['scaler']

for idx, test_df in enumerate(test_dfs):
    y_hat_cp = cp_model.predict_proba(cp_scaler.transform(test_df[models['cp_model']['features']]))[:, 1]
    y_true_cp = test_df['completion']
    accuracy_cp = accuracy_score(y_true_cp, y_hat_cp.round())
    auc_cp = roc_auc_score(y_true_cp, y_hat_cp)
    tn, fp, fn, tp = confusion_matrix(y_true_cp, y_hat_cp.round()).ravel()
    npv_cp = tn / (tn + fn) if (tn + fn) > 0 else 0

    # Calculate PPV (Positive Predictive Value)
    ppv_cp = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    y_hat_fv = fv_model.predict_proba(fv_scaler.transform(test_df[models['fv_model']['features']]))[:, 1]
    y_true_fv = test_df['point_outcome']
    accuracy_fv = accuracy_score(y_true_fv, y_hat_fv.round())
    auc_fv = roc_auc_score(y_true_fv, y_hat_fv)
    tn, fp, fn, tp = confusion_matrix(y_true_fv, y_hat_fv.round()).ravel()
    npv_fv = tn / (tn + fn) if (tn + fn) > 0 else 0

    # Calculate PPV (Positive Predictive Value)
    ppv_fv = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    fv_results.append({
        'Dataset': ['Random', 'Temporal', 'Player'][idx],
        'Accuracy': round(accuracy_fv, 3),
        'AUC': round(auc_fv, 3),
        'PPV': round(ppv_fv, 3),
        'NPV': round(npv_fv, 3)
    })
    cp_results.append({
        'Dataset': ['Random', 'Temporal', 'Player'][idx],
        'Accuracy': round(accuracy_cp, 3),
        'AUC': round(auc_cp, 3),
        'PPV': round(ppv_cp, 3),
        'NPV': round(npv_cp, 3),
    })

cp_results_df = pd.DataFrame(cp_results)
cp_results_df.set_index('Dataset', inplace=True)
fv_results_df = pd.DataFrame(fv_results)
fv_results_df.set_index('Dataset', inplace=True)

cp_results_df.to_csv('./results/cp_results.csv')
cp_results_df

,Accuracy,AUC,PPV,NPV
Dataset,,,,
Random,0.946,0.906,0.947,0.905
Temporal,0.948,0.900,0.949,0.903
Player,0.950,0.913,0.951,0.933


In [12]:
fv_results_df.to_csv('./results/fv_results.csv')
fv_results_df

,Accuracy,AUC,PPV,NPV
Dataset,,,,
Random,0.769,0.791,0.769,0.765
Temporal,0.760,0.787,0.759,0.762
Player,0.789,0.789,0.792,0.773


In [ ]:

baseline_fv_data_config = {'features': thrower_context, 'target':'point_outcome', 'train_df_raw':train_df, 'test_dfs_raw':test_dfs}
baseline_fv_best_models, baseline_fv_best_params, baseline_fv_data_config_updated = run_optuna_trials(baseline_fv_data_config, model_config, 'fv', delete_models=True, storage='sqlite:///model_tuning.db')
metrics_df = evaluate_models(baseline_fv_best_models, baseline_fv_data_config_updated)
metrics_df

[I 2024-11-19 18:50:05,497] A new study created in RDB with name: logreg_fv
[I 2024-11-19 18:50:09,084] Trial 0 finished with value: 0.6236657035964427 and parameters: {'noise_factor': 0.033410321211686876}. Best is trial 0 with value: 0.6236657035964427.


,Model,train_AUC,train_Accuracy,train_NPV,train_PPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_random_PPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_time_PPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV,test_df_thrower_PPV
0,logreg,0.623666,0.706385,0.470879,0.707208,0.622274,0.70795,0.512,0.708562,0.629164,0.694283,0.446009,0.695854,0.606734,0.729637,0.481928,0.73042


In [19]:

baseline_cp_data_config = {'features': thrower_context+receiver_context, 'target':'completion', 'train_df_raw':train_df, 'test_dfs_raw':test_dfs}
baseline_cp_best_models, baseline_cp_best_params, baseline_cp_data_config_updated = run_optuna_trials(baseline_cp_data_config, model_config, 'cp', delete_models=True, storage='sqlite:///model_tuning.db')
metrics_df = evaluate_models(baseline_cp_best_models, baseline_cp_data_config_updated)
metrics_df

[I 2024-11-19 18:51:33,717] A new study created in RDB with name: logreg_cp
[I 2024-11-19 18:51:38,832] Trial 0 finished with value: 0.7279205043871134 and parameters: {'noise_factor': 0.010353511658738485}. Best is trial 0 with value: 0.7279205043871134.


,Model,train_AUC,train_Accuracy,train_NPV,train_PPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_random_PPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_time_PPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV,test_df_thrower_PPV
0,logreg,0.727925,0.93089,0.443032,0.934748,0.726452,0.932088,0.453901,0.935469,0.712662,0.933739,0.423313,0.938698,0.755371,0.935765,0.460993,0.940897
